In [6]:
# IMPORTS

import pandas as pd
import numpy as np

In [7]:
# READ DATASETS

WB_Data_df = pd.read_excel("WBData.xlsx", sheet_name="Data")
WB_Country_df = pd.read_excel("WBData.xlsx", sheet_name="Country")
ESG_Data_df = pd.read_excel("ESGEXCEL.xlsx", sheet_name="Data")
CA_Data_df = pd.read_excel("RawClimateAgreements.xlsx")



In [8]:
# PREPROCESSING DATASETS

WB_Data_df = WB_Data_df.replace('..', "NULL")
WB_Country_df = WB_Country_df.replace('..', "NULL")
ESG_Data_df = ESG_Data_df.replace('..', "NULL")
CA_Data_df = CA_Data_df.fillna('NULL')

ESG_Data_df = ESG_Data_df.set_index('Country Code')
WB_Country_df = WB_Country_df.set_index('Country code')
CA_Data_df = CA_Data_df.set_index('Country')

WB_Country_df = WB_Country_df[WB_Country_df['Region'] != 'Aggregates']

WB_Data_df = WB_Data_df.set_index('Country code')


In [9]:
# COUNTRY entity

Code_ser = WB_Country_df.index
Name_ser = WB_Country_df['Country name']
RegionName_ser = WB_Country_df['Region']
CapitalCity_ser = WB_Country_df['Capital city']
IncomeGroup_ser = WB_Country_df['Income group']
GovernmentEffectiveness_ser = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Government Effectiveness: Estimate']['2022']
PopDens_ser = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Population density (people per sq. km of land area)']['2021']
DevelopedOrDeveloping_ser = WB_Country_df['DevelopedOrDeveloping']
CountryGroup_ser = WB_Country_df['Trade Bloc']

country_df = pd.DataFrame(columns=['CCode', 'Name', 'Region', 'Capital City', 'Income Group', 'Govt. Effectiveness', 'Population Density', 'Economic Status', 'TradeBloc'])
country_SQL_inserts = []

ctrset = set()

for ctrCd in Code_ser:
    CD = ctrCd
    NM = Name_ser[ctrCd]
    RG = RegionName_ser.get(ctrCd, "NULL")
    CC = CapitalCity_ser.get(ctrCd, "NULL")
    IG = IncomeGroup_ser.get(ctrCd, "NULL")
    GE = GovernmentEffectiveness_ser.get(ctrCd, "NULL")
    PD = PopDens_ser.get(ctrCd, "NULL")
    DOD = DevelopedOrDeveloping_ser.get(ctrCd, "NULL")
    CG = CountryGroup_ser.get(ctrCd, "NULL")

    country_df.loc[len(country_df.index)] = [CD, NM, RG, CC, IG, GE, PD, DOD, CG]
    ctrset.add(CD)

    attrs = "CountryCode, CountryName, Region, CapitalCity, IncomeGroup, GovernmentEffectiveness, PopulationDensity, DevelopedOrDeveloping, TradeBloc"
    values  = f"'{CD}', '{NM}', '{RG}', '{CC}', '{IG}', {GE}, {PD}, '{DOD}', '{CG}'"
    SQL_str = f"INSERT INTO country ({attrs}) VALUES ({values});"
    country_SQL_inserts.append(SQL_str)

country_df = country_df.fillna(value='NULL')
country_df.to_excel("Country_Data_RAW.xlsx")

country_SQL_inserts = [ins.replace('nan', 'NULL') for ins in country_SQL_inserts]
country_SQL_inserts = [ins.replace("'NULL'", 'NULL') for ins in country_SQL_inserts]
country_SQL_df = pd.DataFrame(country_SQL_inserts)
country_SQL_df.to_excel("Country_Data_SQL.xlsx")

In [10]:
# CLIMATE DATA entity

co2_filt = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'CO2 emissions (metric tons per capita)']
gini_filt = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Gini index']
tcl_filt = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Tree Cover Loss (hectares)']
renewtar_ser = WB_Data_df[WB_Data_df['Series name'] == 'Renewable energy target'][2011]
renew_filt = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Renewable electricity output (% of total electricity output)']
renewcon_filt = ESG_Data_df[ESG_Data_df['Indicator Name'] == 'Renewable energy consumption (% of total final energy consumption)']


# renewtar_ser = renewtar_ser.rename(2011, 'Renew_Target')

merged_df = 0
merged_df = pd.merge(co2_filt, gini_filt, left_index=True, right_index=True)
merged_df['Indicator Name_x'] = co2_filt['Indicator Name']
merged_df['Indicator Name_y'] = gini_filt['Indicator Name']
tmp = pd.merge(renew_filt, tcl_filt, left_index=True, right_index=True)
tmp['Indicator Name_x'] = renew_filt['Indicator Name']
tmp['Indicator Name_y'] = tcl_filt['Indicator Name']
merged_df = pd.merge(merged_df, tmp, left_index=True, right_index=True)
merged_df = pd.merge(merged_df, renewcon_filt, left_index=True, right_index=True)
merged_df = pd.merge(merged_df, renewtar_ser, left_index=True, right_index=True)

merged_df.loc[merged_df[2011] != 'Yes', 2011] = 'No'

climate_df = pd.DataFrame(columns=['CCode', 'Year', 'Renew energy target', 'Renew energy output(%)', 'Renew energy consump(%)', 'CO2', 'Gini', 'Tree cover loss'])
climate_SQL_inserts = []
for i in range(2008, 2024):
    for ctr in merged_df.index:
        if ctr in ctrset:
            YEAR = i
            RET = merged_df.loc[ctr][2011]
            REO = merged_df.loc[ctr][str(i)+'_x_y']
            REC = merged_df.loc[ctr][str(i)]
            CO2 = merged_df.loc[ctr][str(i)+'_x_x']
            GIN = merged_df.loc[ctr][str(i)+'_y_x']
            TCL = merged_df.loc[ctr][str(i)+'_y_y']

            climate_df.loc[len(climate_df.index)] = [ctr, YEAR, RET, REO, REC, CO2, GIN, TCL]
            
            attrs = "CountryCode, Year, RenewableEnergyTarget, RenewableEnergyOutput, RenewableEnergyConsumption, CO2, GINI, TreeCoverLoss"
            values  = f"'{ctr}', {YEAR}, '{RET}', {REO}, {REC}, {CO2}, {GIN}, {TCL}"
            SQL_str = f"INSERT INTO climatedata ({attrs}) VALUES ({values});"
            climate_SQL_inserts.append(SQL_str)

climate_df = climate_df.fillna(value='NULL')
climate_df.to_excel("Climate_Data_RAW.xlsx")

climate_SQL_inserts = [ins.replace('nan', 'NULL') for ins in climate_SQL_inserts]
Climate_SQL_df = pd.DataFrame(climate_SQL_inserts)
Climate_SQL_df.to_excel("Climate_Data_SQL.xlsx")

In [ ]:
# CLIMATE AGREEMENT DATA entity

ClimateAgreement_inserts = []

ctrcdMap = {WB_Country_df.loc[ctrcd]['Country name']: ctrcd for ctrcd in list(set(WB_Country_df.index))}
ctrcdMap['Costa-rica'] = 'CRI'

CountryName_df = CA_Data_df.index
AgreementName30_df = ['Paris']*len(CountryName_df)
DateSigned30_df = CA_Data_df['2030_Target_Last_Updated']
Target30_df = CA_Data_df['NDC_Target_Text']
TargetDate30_df = ['2030']*len(CountryName_df)

AgreementName50_df = ['Net Zero']*len(CountryName_df)
DateSigned50_df = CA_Data_df['2050_Target_Last_Updated']
Target50_df = CA_Data_df['Net_Zero_Target_Text']
TargetDate50_df = ['2050']*len(CountryName_df)

climateAG_df = pd.DataFrame(columns=['CCode', 'CName', 'AGName', 'Date', 'Target', 'TargetDate'])
for ctrNm in CountryName_df:
    if ctrNm in ctrcdMap:
        CTRCD = ctrcdMap[ctrNm]
        AGR30 = 'Paris'
        DS30 = DateSigned30_df.get(ctrNm, "NULL") 
        T30 = Target30_df.get(ctrNm, "NULL") 
        TD30 = '2030'

        AGR50 = 'Net Zero'
        DS50 = DateSigned50_df.get(ctrNm, "NULL") 
        T50 = Target50_df.get(ctrNm, "NULL") 
        TD50 = '2050'

        climateAG_df.loc[len(climateAG_df.index)] = [CTRCD, ctrNm, AGR30, DS30, T30, TD30]
        climateAG_df.loc[len(climateAG_df.index)] = [CTRCD, ctrNm, AGR50, DS50, T50, TD50]

        attrs = "CountryCode, CountryName, AgreementName, DateSigned, Target , TargetYear"
        val30 = f"'{CTRCD}', '{ctrNm}', '{AGR30}', {DS30}, '{T30}', {TD30}"
        val50 = f"'{CTRCD}', '{ctrNm}', '{AGR50}', {DS50}, '{T50}', {TD50}"
        ClimateAgreement_inserts.append(f"INSERT INTO ClimateAgreement ({attrs}) VALUES ({val30});")
        ClimateAgreement_inserts.append(f"INSERT INTO ClimateAgreement ({attrs}) VALUES ({val50});")

climateAG_df = climateAG_df.fillna(value='NULL')
climateAG_df.to_excel("ClimateAgreement_Data_RAW.xlsx")

ClimateAgreement_inserts = [ins.replace("'NULL'", "NULL") for ins in ClimateAgreement_inserts]  
ClimateAgreement_inserts = [ins.replace("00:00:00", "") for ins in ClimateAgreement_inserts]  
ClimateAgreement_df = pd.DataFrame(ClimateAgreement_inserts)
ClimateAgreement_df.to_excel("ClimateAgreement_Data_SQL.xlsx")